In [ ]:
# install req pkgs: torch, datasets, transformers, scikit-learn, numpy, wandb, sae_lens
!pip install torch datasets transformers scikit-learn numpy wandb sae_lens

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━ 446.8/664.8 MB 51.7 MB/s eta 0:00:05

In [ ]:
import numpy as np
import torch
import random
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
import wandb
import matplotlib.pyplot as plt
from sae_lens import SAE

# set seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
random.seed(42)

# init wandb (mlflow commented out)
wandb.init(project="activation_steering_experiment", config={
    "model": "google/gemma-2b",
    "dataset": "sst2 (glue)",
    "exp": "act steer w/ gemma saes & 50-token rollout"
})


In [ ]:
# login to hf (uncomment and enter token if needed)
#!huggingface-cli login

In [ ]:
# load gemma model & tokenizer
model_name = "google/gemma-2b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, output_hidden_states=True)
model.eval()

# load sae from gemma-scope release (actual id, no placeholder)
# this loads the sae for layer0 with width 16k from the canonical release
sae, cfg_dict, sparsity = SAE.from_pretrained(
    release="gemma-scope-2b-pt-res-canonical",
    sae_id="layer_0/width_16k/canonical",
)
print("loaded sae: cfg:", cfg_dict, "sparsity:", sparsity)

# load sst2 dataset (glue)
dataset = load_dataset("glue", "sst2")
n_samples = 200
train_sentences = dataset['train']['sentence'][:n_samples]
train_labels = dataset['train']['label'][:n_samples]
print(f"using {len(train_sentences)} samples for exp")


In [ ]:
# use layer0 for sae encoding
layer_index = 0

# extract hidden state with sae encoding: tokenize, get layer0, pass thru sae, avg tokens
def extract_hidden_state_with_sae(text):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs, output_hidden_states=True)
    # get hidden states from layer_index
    hidden = outputs.hidden_states[layer_index]
    # pass through sae: returns sparse representation with same shape
    sae_out = sae.encode(hidden)
    # avg over token seq (dim=1)
    rep = sae_out.mean(dim=1).squeeze().detach().numpy()
    return rep

# get sae-encoded hidden states for all train samples
hidden_states = []
for txt in train_sentences:
    try:
        h = extract_hidden_state_with_sae(txt)
        hidden_states.append(h)
    except Exception as e:
        print("err on txt:", txt, e)
hidden_states = np.array(hidden_states)
labels = np.array(train_labels)
print("extracted sae-hidden shape:", hidden_states.shape)

In [ ]:
# train linear probe on sae-hidden states (logistic regression)
clf = LogisticRegression(max_iter=1000)
clf.fit(hidden_states, labels)
C = clf.coef_.flatten()  # learned concept vector
print("trained probe, c shape:", C.shape)

In [ ]:
# extract positive sentiment token unembedding vectors
positive_tokens = ["positive", "good", "great", "amazing", "excellent"]
W_pos_vectors = []
for tok in positive_tokens:
    tok_id = tokenizer.encode(tok)[0]
    vec = model.transformer.wte.weight[tok_id].detach().numpy()
    W_pos_vectors.append(vec)
W_pos_vectors = np.stack(W_pos_vectors)
print("pos token unemb shape:", W_pos_vectors.shape)

# aggregate: mean & pca (pc1)
W_pos_mean = np.mean(W_pos_vectors, axis=0)
pca = PCA(n_components=1)
pca.fit(W_pos_vectors)
W_pos_pc1 = pca.components_[0]

In [ ]:
# cosine sim func
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a)*np.linalg.norm(b))

sim_mean = cosine_similarity(C, W_pos_mean)
sim_pc1 = cosine_similarity(C, W_pos_pc1)
print("cosine sim: c & w_pos_mean:", sim_mean)
print("cosine sim: c & w_pos_pc1:", sim_pc1)

In [ ]:
# act steer intervention: add steering vector to avg sae-hidden state of text, then decode next token
def intervene_and_generate(text, steering_vector, alpha=1.0):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs, output_hidden_states=True)
    # use final layer hidden state avg (using sae encoding optional, here use raw for generation)
    hidden = outputs.hidden_states[-1].mean(dim=1).squeeze().detach()
    hidden_mod = hidden + alpha * torch.tensor(steering_vector, dtype=hidden.dtype)
    logits = hidden_mod @ model.transformer.wte.weight.T
    next_id = torch.argmax(logits).item()
    next_tok = tokenizer.decode([next_id])
    return next_tok

In [ ]:
sample_text = "the movie was"
gen_C = intervene_and_generate(sample_text, C, alpha=1.0)
gen_Wpos = intervene_and_generate(sample_text, W_pos_mean, alpha=1.0)
print("gen token w/ c:", gen_C)
print("gen token w/ w_pos_mean:", gen_Wpos)

In [ ]:
# compute controlled activation attribution (caa) diff for positive tokens
def compute_caa(text, concept_vector, alpha=1.0):
    inputs = tokenizer(text, return_tensors="pt")
    base_out = model(**inputs, output_hidden_states=True)
    base_hidden = base_out.hidden_states[-1].mean(dim=1).squeeze().detach()
    base_logits = base_hidden @ model.transformer.wte.weight.T
    base_probs = torch.softmax(base_logits, dim=-1)

    mod_hidden = base_hidden + alpha * torch.tensor(concept_vector, dtype=base_hidden.dtype)
    mod_logits = mod_hidden @ model.transformer.wte.weight.T
    mod_probs = torch.softmax(mod_logits, dim=-1)

    diffs = {}
    for tok in positive_tokens:
        tok_id = tokenizer.encode(tok)[0]
        diffs[tok] = mod_probs[tok_id].item() - base_probs[tok_id].item()
    return diffs

caa_diffs = compute_caa(sample_text, C, alpha=1.0)
print("caa diffs:", caa_diffs)

# plot caa diffs via wandb
fig, ax = plt.subplots()
ax.bar(list(caa_diffs.keys()), list(caa_diffs.values()))
ax.set_ylabel("prob diff")
ax.set_title("caa: change in token probs")
plt.tight_layout()
wandb.log({"caa_plot": wandb.Image(fig)})
plt.show()

In [ ]:
# 50-token rollout (mdp sim)
def generate_rollout(prompt, steering_vector=None, alpha=1.0, length=50):
    gen_toks = []
    cur_prompt = prompt
    for i in range(length):
        inputs = tokenizer(cur_prompt, return_tensors="pt")
        outputs = model(**inputs, output_hidden_states=True)
        hidden = outputs.hidden_states[-1].mean(dim=1).squeeze().detach()
        if steering_vector is not None:
            hidden = hidden + alpha * torch.tensor(steering_vector, dtype=hidden.dtype)
        logits = hidden @ model.transformer.wte.weight.T
        next_id = torch.argmax(logits).item()
        next_tok = tokenizer.decode([next_id])
        gen_toks.append(next_tok)
        cur_prompt += next_tok
    return "".join(gen_toks)

baseline_rollout = generate_rollout("the movie was", steering_vector=None, length=50)
intervened_rollout = generate_rollout("the movie was", steering_vector=C, alpha=1.0, length=50)
print("baseline rollout:\n", baseline_rollout)
print("\nintervened rollout (w/ c):\n", intervened_rollout)

wandb.log({
    "baseline_rollout": baseline_rollout,
    "intervened_rollout": intervened_rollout
})

wandb.finish()
print("exp logged w/ wandb")
